In [1]:
import pandas as pd
df = pd.read_excel("final data 11-23.xlsx")
df.head()

,game_id,first,time_control_name,game_end_reason,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,...,Player_rack_used_rate,Bot_win,Player_win,Bot_game,Player_game,Bot_score,Player_score,Bot_rating,Player_rating,created_at
0,43828,HastyBot,ultrablitz,RESIGNED,CSW21,15,0,RATED,1,61.405180,...,0.178571,1,0,14112,1,472,15,2148,1979.0,2022-09-11 13:06:11
1,5765,0927399533,regular,TIME,CSW21,1200,0,RATED,1,1268.252752,...,0.482143,1,0,4847,1,462,141,2012,1447.0,2022-08-09 14:25:28
2,40264,BetterBot,regular,TIME,CSW21,1200,0,RATED,1,1262.786704,...,0.467532,1,0,4871,2,356,212,1966,1408.0,2022-08-09 15:26:54
3,44376,0927399533,regular,STANDARD,CSW21,1200,0,RATED,1,1209.746654,...,0.396825,1,0,5327,3,535,227,2090,1372.0,2022-08-10 16:28:43
4,32633,BetterBot,regular,STANDARD,CSW21,1200,0,RATED,1,1231.166683,...,0.519048,0,1,5336,4,383,391,2114,1367.0,2022-08-10 16:49:00


In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_excel("hist_player_score_final_data_11-23 with board metrics.xlsx")

In [3]:
df_sorted = df.sort_values(by=['Player_nickname', 'created_at']).groupby('Player_nickname')
df_sorted.head(3)

,game_id,first,time_control_name,game_end_reason,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,Bot_nickname,Player_nickname,Bot_turns_count,Player_turns_count,Bot_max_point,Player_max_point,Bot_min_point,Player_min_point,Bot_avg_bingo,Player_avg_bingo,Bot_word_len,Player_word_len,Bot_count_letters_JQXZ,Player_count_letters_JQXZ,Bot_rack_used_rate,Player_rack_used_rate,Bot_win,Player_win,Bot_game,Player_game,Bot_score,Player_score,Bot_rating,Player_rating,created_at,hist_player_score,Bot_Double_Word,Bot_Triple_Word,Bot_Double_Letter,Bot_Triple_Letter,Player_Double_Word,Player_Triple_Word,Player_Double_Letter,Player_Triple_Letter
0,43828,HastyBot,ultrablitz,RESIGNED,CSW21,15,0,RATED,1,61.405180,HastyBot,0188889876,16,16,72,14,0,-10,0.062500,0.000000,4.187500,0.312500,2,0,0.638095,0.178571,1,0,14112,1,472,15,2148,1979.0,2022-09-11 13:06:11,1700.0,4,5,8,4,0,0,1,1
1,5765,0927399533,regular,TIME,CSW21,1200,0,RATED,1,1268.252752,BetterBot,0927399533,11,12,92,35,16,-10,0.272727,0.000000,4.090909,2.250000,4,0,0.584416,0.482143,1,0,4847,1,462,141,2012,1447.0,2022-08-09 14:25:28,1700.0,2,3,7,5,4,2,3,0
2,40264,BetterBot,regular,TIME,CSW21,1200,0,RATED,1,1262.786704,BetterBot,0927399533,12,12,74,36,8,-10,0.166667,0.000000,4.166667,3.000000,0,2,0.595238,0.467532,1,0,4871,2,356,212,1966,1408.0,2022-08-09 15:26:54,1691.2,7,0,4,6,3,4,5,5
3,44376,0927399533,regular,STANDARD,CSW21,1200,0,RATED,1,1209.746654,BetterBot,0927399533,15,16,102,75,8,-10,0.125000,0.062500,3.750000,2.437500,4,0,0.625397,0.396825,1,0,5327,3,535,227,2090,1372.0,2022-08-10 16:28:43,1677.8,6,4,9,5,4,3,9,0
20,47364,1,regular,STANDARD,NWL20,1200,0,CASUAL,1,372.547459,STEEBot,1,11,11,83,72,8,15,0.166667,0.181818,4.000000,4.454545,2,2,0.675325,0.642857,1,0,11834,1,403,385,1901,1632.0,2022-08-25 03:33:16,1700.0,6,2,5,2,4,2,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72763,31179,zuzu,regular,STANDARD,CSW21,1200,0,RATED,1,817.400729,BetterBot,zuzu,14,15,94,60,0,6,0.071429,0.062500,3.000000,3.437500,2,2,0.461538,0.561905,0,1,12354,2,328,410,1975,1850.0,2022-08-27 06:39:13,1686.9,8,1,8,2,3,5,5,5
72764,10944,BetterBot,regular,STANDARD,CSW21,1200,0,RATED,1,640.984386,BetterBot,zuzu,14,14,86,75,0,4,0.142857,0.066667,3.500000,3.266667,4,0,0.538462,0.551020,1,0,12358,3,504,340,1952,1870.0,2022-08-27 06:53:05,1754.4,5,2,8,3,5,4,6,1
72769,66022,zyzzyva,regular,STANDARD,NWL20,1380,0,RATED,1,626.274697,STEEBot,zyzzyva,13,13,78,66,0,0,0.142857,0.076923,3.285714,3.846154,3,1,0.649351,0.595238,1,0,573,1,393,357,1859,1641.0,2022-07-28 18:47:59,1700.0,5,1,5,2,6,3,4,1
72770,2581,STEEBot,regular,STANDARD,NWL20,1380,0,RATED,1,775.765387,STEEBot,zyzzyva,12,11,70,76,13,17,0.076923,0.181818,3.923077,4.000000,3,1,0.642857,0.571429,0,1,579,2,358,438,1863,1637.0,2022-07-28 18:58:36,1675.9,5,2,6,5,3,4,6,5


In [6]:
# Creating historical average rating
import queue
import statistics


def historical_average_rating(df_sorted):
    base_rating = 1700
    
    score_df = pd.DataFrame(columns=['hist_player_score', 'game_id'])
    #Iterating through the games
    counter = 0
    group_count = 0
    games = 10
    for name, group_df in df_sorted:
        counter += 1

        # Initializing variables
        win = 0
        loss = 0
        games_played = 0
        
        # Creating a new queue
        oar_queue = queue.Queue()
        wins_queue = queue.Queue()
        for i in range(10):
            oar_queue.put(1700)
            wins_queue.put(0)
        
        for index, row in group_df.iterrows():
            
            # Getting details for 400(w-l)/g
            games_played = row['Player_game'] - 1
            
            
            
            # Adding fake games when less than 10, to have data
            if games_played == 0:
                fake_games = 10
                win_fake = 5
                loss_fake = 5
                win = 0
                loss = 0
            elif games_played < 10:
                fake_games = games-games_played
                win_fake = fake_games/2
                loss_fake = fake_games/2
                prev_10_win = get_prev_10_wins(wins_queue)
                prev_10_loss = games_played-prev_10_win
                win = prev_10_win + win_fake
                loss = prev_10_loss + loss_fake
            else:
                prev_10_win = get_prev_10_wins(wins_queue)
                prev_10_loss = games-prev_10_win
                win = prev_10_win
                loss = prev_10_loss
            
            #print(index)
            oar_score = get_oar_score(oar_queue)
            # Doing the calculation
            hist_score = oar_score + ((400)*(win-loss))/games

            score_df = pd.concat([score_df, pd.DataFrame({'hist_player_score': [hist_score], 'game_id': [row['game_id']]})], ignore_index=True)
            # After current game, update the variable with current game details
                # Updating bot rating, win/loss 
            oar_queue.put(row['Bot_rating'])
            removed_element = oar_queue.get()
            wins_queue.put(row['Player_win'])
            removed_element = wins_queue.get()
            
            
    return score_df
def get_oar_score(queue):
    
    values_list = list(queue.queue)
    oar = statistics.mean(values_list)
    return oar
    #opp_avg_rating = get_opp_avg_rating()

def get_prev_10_wins(wins_queue):
    result_sum = 0
    result_sum = sum(item for item in wins_queue.queue)
    
    return result_sum


In [11]:
# Creating historical average rating
import queue
import statistics


def historical_average_rating_20(df_sorted):
    base_rating = 1700
    
    score_df = pd.DataFrame(columns=['hist_player_score_20', 'game_id'])
    #Iterating through the games
    counter = 0
    group_count = 0
    games = 20
    for name, group_df in df_sorted:
        counter += 1

        # Initializing variables
        win = 0
        loss = 0
        games_played = 0
        
        # Creating a new queue
        oar_queue = queue.Queue()
        wins_queue = queue.Queue()
        for i in range(20):
            oar_queue.put(1700)
            wins_queue.put(0)
        
        for index, row in group_df.iterrows():
            
            # Getting details for 400(w-l)/g
            games_played = row['Player_game'] - 1
            
            
            
            # Adding fake games when less than 10, to have data
            if games_played == 0:
                fake_games = 20
                win_fake = 10
                loss_fake = 10
                win = 0
                loss = 0
            elif games_played < 20:
                fake_games = games-games_played
                win_fake = fake_games/2
                loss_fake = fake_games/2
                prev_20_win = get_prev_20_wins(wins_queue)
                prev_20_loss = games_played-prev_20_win
                win = prev_20_win + win_fake
                loss = prev_20_loss + loss_fake
            else:
                prev_10_win = get_prev_20_wins(wins_queue)
                prev_10_loss = games-prev_20_win
                win = prev_20_win
                loss = prev_20_loss
            
            #print(index)
            oar_score = get_oar_score(oar_queue)
            # Doing the calculation
            hist_score = oar_score + ((400)*(win-loss))/games

            score_df = pd.concat([score_df, pd.DataFrame({'hist_player_score_20': [hist_score], 'game_id': [row['game_id']]})], ignore_index=True)
            # After current game, update the variable with current game details
                # Updating bot rating, win/loss 
            oar_queue.put(row['Bot_rating'])
            removed_element = oar_queue.get()
            wins_queue.put(row['Player_win'])
            removed_element = wins_queue.get()
            
            
    return score_df
def get_oar_score(queue):
    
    values_list = list(queue.queue)
    oar = statistics.mean(values_list)
    return oar
    #opp_avg_rating = get_opp_avg_rating()

def get_prev_20_wins(wins_queue):
    result_sum = 0
    result_sum = sum(item for item in wins_queue.queue)
    
    return result_sum


In [24]:
def get_previous_win_rate(df_sorted):
    
    last_win_df = pd.DataFrame(columns=['last_game_win_flag', 'game_id'])
    for name, group_df in df_sorted:
        for index, row in group_df.iterrows():
            last_win = 0
            current_game_number = row['Player_game']
            if current_game_number > 1:
                previous_game_number = current_game_number - 1
                previous_game_row = group_df[group_df['Player_game'] == previous_game_number]
                if not previous_game_row.empty and previous_game_row['Player_win'].iloc[0] == 1:
                    print(f"For player in game {current_game_number}, won the previous game ({previous_game_number}).")
                    last_win = 1
            last_win_df = pd.concat([last_win_df, pd.DataFrame({'last_game_win_flag': [last_win], 'game_id': [row['game_id']]})], ignore_index=True)
    return last_win_df
last_win = get_previous_win_rate(df_sorted)   
last_win.head()

For player in game 5, won the previous game (4).
For player in game 7, won the previous game (6).
For player in game 15, won the previous game (14).
For player in game 22, won the previous game (21).
For player in game 30, won the previous game (29).
For player in game 18, won the previous game (17).
For player in game 23, won the previous game (22).
For player in game 29, won the previous game (28).
For player in game 37, won the previous game (36).
For player in game 2, won the previous game (1).
For player in game 6, won the previous game (5).
For player in game 7, won the previous game (6).
For player in game 9, won the previous game (8).
For player in game 12, won the previous game (11).
For player in game 14, won the previous game (13).
For player in game 15, won the previous game (14).
For player in game 17, won the previous game (16).
For player in game 21, won the previous game (20).
For player in game 24, won the previous game (23).
For player in game 25, won the previous gam

For player in game 38, won the previous game (37).
For player in game 3, won the previous game (2).
For player in game 5, won the previous game (4).
For player in game 8, won the previous game (7).
For player in game 9, won the previous game (8).
For player in game 11, won the previous game (10).
For player in game 13, won the previous game (12).
For player in game 15, won the previous game (14).
For player in game 16, won the previous game (15).
For player in game 19, won the previous game (18).
For player in game 25, won the previous game (24).
For player in game 26, won the previous game (25).
For player in game 31, won the previous game (30).
For player in game 32, won the previous game (31).
For player in game 34, won the previous game (33).
For player in game 36, won the previous game (35).
For player in game 42, won the previous game (41).
For player in game 46, won the previous game (45).
For player in game 48, won the previous game (47).
For player in game 50, won the previous

For player in game 12, won the previous game (11).
For player in game 14, won the previous game (13).
For player in game 15, won the previous game (14).
For player in game 16, won the previous game (15).
For player in game 20, won the previous game (19).
For player in game 21, won the previous game (20).
For player in game 23, won the previous game (22).
For player in game 24, won the previous game (23).
For player in game 26, won the previous game (25).
For player in game 28, won the previous game (27).
For player in game 29, won the previous game (28).
For player in game 30, won the previous game (29).
For player in game 37, won the previous game (36).
For player in game 38, won the previous game (37).
For player in game 41, won the previous game (40).
For player in game 44, won the previous game (43).
For player in game 45, won the previous game (44).
For player in game 46, won the previous game (45).
For player in game 47, won the previous game (46).
For player in game 49, won the 

For player in game 151, won the previous game (150).
For player in game 157, won the previous game (156).
For player in game 159, won the previous game (158).
For player in game 164, won the previous game (163).
For player in game 174, won the previous game (173).
For player in game 183, won the previous game (182).
For player in game 189, won the previous game (188).
For player in game 195, won the previous game (194).
For player in game 213, won the previous game (212).
For player in game 214, won the previous game (213).
For player in game 215, won the previous game (214).
For player in game 217, won the previous game (216).
For player in game 219, won the previous game (218).
For player in game 222, won the previous game (221).
For player in game 225, won the previous game (224).
For player in game 226, won the previous game (225).
For player in game 239, won the previous game (238).
For player in game 251, won the previous game (250).
For player in game 253, won the previous game 

For player in game 22, won the previous game (21).
For player in game 23, won the previous game (22).
For player in game 24, won the previous game (23).
For player in game 26, won the previous game (25).
For player in game 27, won the previous game (26).
For player in game 28, won the previous game (27).
For player in game 29, won the previous game (28).
For player in game 30, won the previous game (29).
For player in game 32, won the previous game (31).
For player in game 33, won the previous game (32).
For player in game 4, won the previous game (3).
For player in game 8, won the previous game (7).
For player in game 12, won the previous game (11).
For player in game 13, won the previous game (12).
For player in game 2, won the previous game (1).
For player in game 4, won the previous game (3).
For player in game 5, won the previous game (4).
For player in game 3, won the previous game (2).
For player in game 4, won the previous game (3).
For player in game 5, won the previous game (

...

For player in game 140, won the previous game (139).
For player in game 141, won the previous game (140).
For player in game 142, won the previous game (141).
For player in game 143, won the previous game (142).
For player in game 144, won the previous game (143).
For player in game 146, won the previous game (145).
For player in game 4, won the previous game (3).
For player in game 3, won the previous game (2).
For player in game 12, won the previous game (11).
For player in game 21, won the previous game (20).
For player in game 26, won the previous game (25).
For player in game 29, won the previous game (28).
For player in game 35, won the previous game (34).
For player in game 47, won the previous game (46).
For player in game 56, won the previous game (55).
For player in game 62, won the previous game (61).
For player in game 64, won the previous game (63).
For player in game 69, won the previous game (68).
For player in game 76, won the previous game (75).
For player in game 2, w

For player in game 93, won the previous game (92).
For player in game 101, won the previous game (100).
For player in game 105, won the previous game (104).
For player in game 109, won the previous game (108).
For player in game 113, won the previous game (112).
For player in game 114, won the previous game (113).
For player in game 116, won the previous game (115).
For player in game 117, won the previous game (116).
For player in game 120, won the previous game (119).
For player in game 126, won the previous game (125).
For player in game 127, won the previous game (126).
For player in game 129, won the previous game (128).
For player in game 133, won the previous game (132).
For player in game 136, won the previous game (135).
For player in game 140, won the previous game (139).
For player in game 144, won the previous game (143).
For player in game 145, won the previous game (144).
For player in game 147, won the previous game (146).
For player in game 156, won the previous game (1

For player in game 7, won the previous game (6).
For player in game 11, won the previous game (10).
For player in game 12, won the previous game (11).
For player in game 14, won the previous game (13).
For player in game 17, won the previous game (16).
For player in game 19, won the previous game (18).
For player in game 22, won the previous game (21).
For player in game 25, won the previous game (24).
For player in game 26, won the previous game (25).
For player in game 27, won the previous game (26).
For player in game 30, won the previous game (29).
For player in game 35, won the previous game (34).
For player in game 36, won the previous game (35).
For player in game 39, won the previous game (38).
For player in game 40, won the previous game (39).
For player in game 43, won the previous game (42).
For player in game 45, won the previous game (44).
For player in game 48, won the previous game (47).
For player in game 49, won the previous game (48).
For player in game 53, won the pr

For player in game 21, won the previous game (20).
For player in game 24, won the previous game (23).
For player in game 28, won the previous game (27).
For player in game 6, won the previous game (5).
For player in game 16, won the previous game (15).
For player in game 20, won the previous game (19).
For player in game 22, won the previous game (21).
For player in game 33, won the previous game (32).
For player in game 37, won the previous game (36).
For player in game 2, won the previous game (1).
For player in game 3, won the previous game (2).
For player in game 6, won the previous game (5).
For player in game 2, won the previous game (1).
For player in game 3, won the previous game (2).
For player in game 4, won the previous game (3).
For player in game 6, won the previous game (5).
For player in game 8, won the previous game (7).
For player in game 11, won the previous game (10).
For player in game 12, won the previous game (11).
For player in game 13, won the previous game (12)

,last_game_win_flag,game_id
0,0,43828
1,0,5765
2,0,40264
3,0,44376
4,0,32633


In [12]:
score = historical_average_rating_20(df_sorted)
score

,hist_player_score_20,game_id
0,1700.00,43828
1,1700.00,5765
2,1695.60,40264
3,1688.90,44376
4,1688.40,32633
...,...,...
72768,1725.35,7264
72769,1700.00,66022
72770,1687.95,2581
72771,1716.10,54204


In [27]:
last_win.head(40)

,last_game_win_flag,game_id
0,0,43828
1,0,5765
2,0,40264
3,0,44376
4,0,32633
5,1,4315
6,0,45148
7,0,65516
8,0,56816
9,0,19582


In [28]:
final_df_win = pd.merge(final_df, last_win, on='game_id', how='left')
final_df_win

,game_id,first,time_control_name,game_end_reason,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,Bot_nickname,Player_nickname,Bot_turns_count,Player_turns_count,Bot_max_point,Player_max_point,Bot_min_point,Player_min_point,Bot_avg_bingo,Player_avg_bingo,Bot_word_len,Player_word_len,Bot_count_letters_JQXZ,Player_count_letters_JQXZ,Bot_rack_used_rate,Player_rack_used_rate,Bot_win,Player_win,Bot_game,Player_game,Bot_score,Player_score,Bot_rating,Player_rating,created_at,hist_player_score,Bot_Double_Word,Bot_Triple_Word,Bot_Double_Letter,Bot_Triple_Letter,Player_Double_Word,Player_Triple_Word,Player_Double_Letter,Player_Triple_Letter,hist_player_score_20,last_game_win_flag
0,43828,HastyBot,ultrablitz,RESIGNED,CSW21,15,0,RATED,1,61.405180,HastyBot,0188889876,16,16,72,14,0,-10,0.062500,0.000000,4.187500,0.312500,2,0,0.638095,0.178571,1,0,14112,1,472,15,2148,1979.0,2022-09-11 13:06:11,1700.0,4,5,8,4,0,0,1,1,1700.00,0
1,5765,0927399533,regular,TIME,CSW21,1200,0,RATED,1,1268.252752,BetterBot,0927399533,11,12,92,35,16,-10,0.272727,0.000000,4.090909,2.250000,4,0,0.584416,0.482143,1,0,4847,1,462,141,2012,1447.0,2022-08-09 14:25:28,1700.0,2,3,7,5,4,2,3,0,1700.00,0
2,40264,BetterBot,regular,TIME,CSW21,1200,0,RATED,1,1262.786704,BetterBot,0927399533,12,12,74,36,8,-10,0.166667,0.000000,4.166667,3.000000,0,2,0.595238,0.467532,1,0,4871,2,356,212,1966,1408.0,2022-08-09 15:26:54,1691.2,7,0,4,6,3,4,5,5,1695.60,0
3,44376,0927399533,regular,STANDARD,CSW21,1200,0,RATED,1,1209.746654,BetterBot,0927399533,15,16,102,75,8,-10,0.125000,0.062500,3.750000,2.437500,4,0,0.625397,0.396825,1,0,5327,3,535,227,2090,1372.0,2022-08-10 16:28:43,1677.8,6,4,9,5,4,3,9,0,1688.90,0
4,32633,BetterBot,regular,STANDARD,CSW21,1200,0,RATED,1,1231.166683,BetterBot,0927399533,15,16,85,85,6,-10,0.066667,0.117647,3.400000,2.823529,3,1,0.520952,0.519048,0,1,5336,4,383,391,2114,1367.0,2022-08-10 16:49:00,1676.8,7,1,6,5,6,5,4,2,1688.40,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72768,7264,zuzu,regular,STANDARD,CSW21,1200,0,RATED,1,683.258274,HastyBot,zuzu,13,14,77,78,10,0,0.076923,0.066667,3.846154,3.200000,3,1,0.549451,0.538462,1,0,15306,7,414,327,2231,1855.0,2022-09-13 09:23:17,1750.7,5,2,5,4,5,5,4,1,1725.35,0
72769,66022,zyzzyva,regular,STANDARD,NWL20,1380,0,RATED,1,626.274697,STEEBot,zyzzyva,13,13,78,66,0,0,0.142857,0.076923,3.285714,3.846154,3,1,0.649351,0.595238,1,0,573,1,393,357,1859,1641.0,2022-07-28 18:47:59,1700.0,5,1,5,2,6,3,4,1,1700.00,0
72770,2581,STEEBot,regular,STANDARD,NWL20,1380,0,RATED,1,775.765387,STEEBot,zyzzyva,12,11,70,76,13,17,0.076923,0.181818,3.923077,4.000000,3,1,0.642857,0.571429,0,1,579,2,358,438,1863,1637.0,2022-07-28 18:58:36,1675.9,5,2,6,5,3,4,6,5,1687.95,0
72771,54204,STEEBot,regular,STANDARD,NWL20,1200,0,RATED,1,624.940244,STEEBot,zyzzyva,14,14,70,73,0,0,0.000000,0.133333,3.571429,3.200000,2,2,0.565934,0.582418,0,1,2119,3,357,412,1856,1626.0,2022-08-01 01:07:41,1732.2,6,3,5,4,9,1,5,4,1716.10,1


In [29]:
final_df_win.isnull().sum()

game_id                          0
first                            0
time_control_name                0
game_end_reason                  0
lexicon                          0
initial_time_seconds             0
increment_seconds                0
rating_mode                      0
max_overtime_minutes             0
game_duration_seconds            0
Bot_nickname                     0
Player_nickname                  0
Bot_turns_count                  0
Player_turns_count               0
Bot_max_point                    0
Player_max_point                 0
Bot_min_point                    0
Player_min_point                 0
Bot_avg_bingo                    0
Player_avg_bingo                 0
Bot_word_len                     0
Player_word_len                  0
Bot_count_letters_JQXZ           0
Player_count_letters_JQXZ        0
Bot_rack_used_rate               0
Player_rack_used_rate            0
Bot_win                          0
Player_win                       0
Bot_game            

In [30]:
final_df_win

,game_id,first,time_control_name,game_end_reason,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,Bot_nickname,Player_nickname,Bot_turns_count,Player_turns_count,Bot_max_point,Player_max_point,Bot_min_point,Player_min_point,Bot_avg_bingo,Player_avg_bingo,Bot_word_len,Player_word_len,Bot_count_letters_JQXZ,Player_count_letters_JQXZ,Bot_rack_used_rate,Player_rack_used_rate,Bot_win,Player_win,Bot_game,Player_game,Bot_score,Player_score,Bot_rating,Player_rating,created_at,hist_player_score,Bot_Double_Word,Bot_Triple_Word,Bot_Double_Letter,Bot_Triple_Letter,Player_Double_Word,Player_Triple_Word,Player_Double_Letter,Player_Triple_Letter,hist_player_score_20,last_game_win_flag
0,43828,HastyBot,ultrablitz,RESIGNED,CSW21,15,0,RATED,1,61.405180,HastyBot,0188889876,16,16,72,14,0,-10,0.062500,0.000000,4.187500,0.312500,2,0,0.638095,0.178571,1,0,14112,1,472,15,2148,1979.0,2022-09-11 13:06:11,1700.0,4,5,8,4,0,0,1,1,1700.00,0
1,5765,0927399533,regular,TIME,CSW21,1200,0,RATED,1,1268.252752,BetterBot,0927399533,11,12,92,35,16,-10,0.272727,0.000000,4.090909,2.250000,4,0,0.584416,0.482143,1,0,4847,1,462,141,2012,1447.0,2022-08-09 14:25:28,1700.0,2,3,7,5,4,2,3,0,1700.00,0
2,40264,BetterBot,regular,TIME,CSW21,1200,0,RATED,1,1262.786704,BetterBot,0927399533,12,12,74,36,8,-10,0.166667,0.000000,4.166667,3.000000,0,2,0.595238,0.467532,1,0,4871,2,356,212,1966,1408.0,2022-08-09 15:26:54,1691.2,7,0,4,6,3,4,5,5,1695.60,0
3,44376,0927399533,regular,STANDARD,CSW21,1200,0,RATED,1,1209.746654,BetterBot,0927399533,15,16,102,75,8,-10,0.125000,0.062500,3.750000,2.437500,4,0,0.625397,0.396825,1,0,5327,3,535,227,2090,1372.0,2022-08-10 16:28:43,1677.8,6,4,9,5,4,3,9,0,1688.90,0
4,32633,BetterBot,regular,STANDARD,CSW21,1200,0,RATED,1,1231.166683,BetterBot,0927399533,15,16,85,85,6,-10,0.066667,0.117647,3.400000,2.823529,3,1,0.520952,0.519048,0,1,5336,4,383,391,2114,1367.0,2022-08-10 16:49:00,1676.8,7,1,6,5,6,5,4,2,1688.40,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72768,7264,zuzu,regular,STANDARD,CSW21,1200,0,RATED,1,683.258274,HastyBot,zuzu,13,14,77,78,10,0,0.076923,0.066667,3.846154,3.200000,3,1,0.549451,0.538462,1,0,15306,7,414,327,2231,1855.0,2022-09-13 09:23:17,1750.7,5,2,5,4,5,5,4,1,1725.35,0
72769,66022,zyzzyva,regular,STANDARD,NWL20,1380,0,RATED,1,626.274697,STEEBot,zyzzyva,13,13,78,66,0,0,0.142857,0.076923,3.285714,3.846154,3,1,0.649351,0.595238,1,0,573,1,393,357,1859,1641.0,2022-07-28 18:47:59,1700.0,5,1,5,2,6,3,4,1,1700.00,0
72770,2581,STEEBot,regular,STANDARD,NWL20,1380,0,RATED,1,775.765387,STEEBot,zyzzyva,12,11,70,76,13,17,0.076923,0.181818,3.923077,4.000000,3,1,0.642857,0.571429,0,1,579,2,358,438,1863,1637.0,2022-07-28 18:58:36,1675.9,5,2,6,5,3,4,6,5,1687.95,0
72771,54204,STEEBot,regular,STANDARD,NWL20,1200,0,RATED,1,624.940244,STEEBot,zyzzyva,14,14,70,73,0,0,0.000000,0.133333,3.571429,3.200000,2,2,0.565934,0.582418,0,1,2119,3,357,412,1856,1626.0,2022-08-01 01:07:41,1732.2,6,3,5,4,9,1,5,4,1716.10,1


In [31]:
final_df_win.to_excel('hist_lastWin_final_data_11-25.xlsx', index=False)

In [32]:
final_df_win

,game_id,first,time_control_name,game_end_reason,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,Bot_nickname,Player_nickname,Bot_turns_count,Player_turns_count,Bot_max_point,Player_max_point,Bot_min_point,Player_min_point,Bot_avg_bingo,Player_avg_bingo,Bot_word_len,Player_word_len,Bot_count_letters_JQXZ,Player_count_letters_JQXZ,Bot_rack_used_rate,Player_rack_used_rate,Bot_win,Player_win,Bot_game,Player_game,Bot_score,Player_score,Bot_rating,Player_rating,created_at,hist_player_score,Bot_Double_Word,Bot_Triple_Word,Bot_Double_Letter,Bot_Triple_Letter,Player_Double_Word,Player_Triple_Word,Player_Double_Letter,Player_Triple_Letter,hist_player_score_20,last_game_win_flag
0,43828,HastyBot,ultrablitz,RESIGNED,CSW21,15,0,RATED,1,61.405180,HastyBot,0188889876,16,16,72,14,0,-10,0.062500,0.000000,4.187500,0.312500,2,0,0.638095,0.178571,1,0,14112,1,472,15,2148,1979.0,2022-09-11 13:06:11,1700.0,4,5,8,4,0,0,1,1,1700.00,0
1,5765,0927399533,regular,TIME,CSW21,1200,0,RATED,1,1268.252752,BetterBot,0927399533,11,12,92,35,16,-10,0.272727,0.000000,4.090909,2.250000,4,0,0.584416,0.482143,1,0,4847,1,462,141,2012,1447.0,2022-08-09 14:25:28,1700.0,2,3,7,5,4,2,3,0,1700.00,0
2,40264,BetterBot,regular,TIME,CSW21,1200,0,RATED,1,1262.786704,BetterBot,0927399533,12,12,74,36,8,-10,0.166667,0.000000,4.166667,3.000000,0,2,0.595238,0.467532,1,0,4871,2,356,212,1966,1408.0,2022-08-09 15:26:54,1691.2,7,0,4,6,3,4,5,5,1695.60,0
3,44376,0927399533,regular,STANDARD,CSW21,1200,0,RATED,1,1209.746654,BetterBot,0927399533,15,16,102,75,8,-10,0.125000,0.062500,3.750000,2.437500,4,0,0.625397,0.396825,1,0,5327,3,535,227,2090,1372.0,2022-08-10 16:28:43,1677.8,6,4,9,5,4,3,9,0,1688.90,0
4,32633,BetterBot,regular,STANDARD,CSW21,1200,0,RATED,1,1231.166683,BetterBot,0927399533,15,16,85,85,6,-10,0.066667,0.117647,3.400000,2.823529,3,1,0.520952,0.519048,0,1,5336,4,383,391,2114,1367.0,2022-08-10 16:49:00,1676.8,7,1,6,5,6,5,4,2,1688.40,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72768,7264,zuzu,regular,STANDARD,CSW21,1200,0,RATED,1,683.258274,HastyBot,zuzu,13,14,77,78,10,0,0.076923,0.066667,3.846154,3.200000,3,1,0.549451,0.538462,1,0,15306,7,414,327,2231,1855.0,2022-09-13 09:23:17,1750.7,5,2,5,4,5,5,4,1,1725.35,0
72769,66022,zyzzyva,regular,STANDARD,NWL20,1380,0,RATED,1,626.274697,STEEBot,zyzzyva,13,13,78,66,0,0,0.142857,0.076923,3.285714,3.846154,3,1,0.649351,0.595238,1,0,573,1,393,357,1859,1641.0,2022-07-28 18:47:59,1700.0,5,1,5,2,6,3,4,1,1700.00,0
72770,2581,STEEBot,regular,STANDARD,NWL20,1380,0,RATED,1,775.765387,STEEBot,zyzzyva,12,11,70,76,13,17,0.076923,0.181818,3.923077,4.000000,3,1,0.642857,0.571429,0,1,579,2,358,438,1863,1637.0,2022-07-28 18:58:36,1675.9,5,2,6,5,3,4,6,5,1687.95,0
72771,54204,STEEBot,regular,STANDARD,NWL20,1200,0,RATED,1,624.940244,STEEBot,zyzzyva,14,14,70,73,0,0,0.000000,0.133333,3.571429,3.200000,2,2,0.565934,0.582418,0,1,2119,3,357,412,1856,1626.0,2022-08-01 01:07:41,1732.2,6,3,5,4,9,1,5,4,1716.10,1


In [16]:
board = pd.read_excel(r"C:\Users\inbox\Downloads\Predictive Project\board_metrics.xlsx")
board

,game_id,Bot_Double_Word,Bot_Triple_Word,Bot_Double_Letter,Bot_Triple_Letter,Player_Double_Word,Player_Triple_Word,Player_Double_Letter,Player_Triple_Letter
0,1,9,2,5,2,3,3,6,3
1,2,6,2,5,6,4,5,4,3
2,3,7,1,8,1,4,3,4,6
3,4,9,3,8,3,4,1,5,2
4,5,6,3,9,4,4,3,6,3
...,...,...,...,...,...,...,...,...,...
72768,72769,5,4,7,3,6,2,2,6
72769,72770,6,3,9,4,3,4,7,1
72770,72771,5,2,6,4,6,3,8,1
72771,72772,7,3,6,2,4,2,5,5


In [20]:
final_df  = pd.merge(final_df, board, on=["game_id"], how="left")

In [21]:
final_df.isnull().sum()

game_id                          0
first                            0
time_control_name                0
game_end_reason                  0
lexicon                          0
initial_time_seconds             0
increment_seconds                0
rating_mode                      0
max_overtime_minutes             0
game_duration_seconds            0
Bot_nickname                     0
Player_nickname                  0
Bot_turns_count                  0
Player_turns_count               0
Bot_max_point                    0
Player_max_point                 0
Bot_min_point                    0
Player_min_point                 0
Bot_avg_bingo                    0
Player_avg_bingo                 0
Bot_word_len                     0
Player_word_len                  0
Bot_count_letters_JQXZ           0
Player_count_letters_JQXZ        0
Bot_rack_used_rate               0
Player_rack_used_rate            0
Bot_win                          0
Player_win                       0
Bot_game            

In [22]:
final_df.to_excel('hist_player_score_final_data_11-23 with board metrics.xlsx', index=False)